# Structured Outputs with Ollama

Open-source LLMS are gaining popularity, and with the release of Ollama's OpenAI compatibility layer, it has become possible to obtain structured outputs using JSON schema.

By the end of this blog post, you will learn how to effectively utilize instructor with Ollama. But before we proceed, let's first explore the concept of patching.

## Patching

Instructor's patch enhances an openai api with the following features:
* `response_model` in `create` calls that returns a pydantic model
* `max_retries` in `create` calls that retries the call if it fails by using a backoff strategy

**Learn More**
To learn more, please refer to the docs. To understand the benefits of using Pydantic with Instructor, visit the tips and tricks section of the why use Pydantic page.

## Ollama

Start by downloading Ollama, and then pull a model such as Llama 3 or Mistral.

**Make sure you update your **`ollama` to the latest version!

In [ ]:
# ollama pull llama3

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor

In [ ]:
from dotenv import load_dotenv
load_dotenv("../api_keys.env")

In [ ]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

In [ ]:
class Character(BaseModel):
    name: str
    age: int
    fact: List[str] = Field(..., description="A list of facts about the character")

In [ ]:
resp = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": "Tell me about the Harry Potter",
        }
    ],
    response_model=Character,
)

In [ ]:
print(resp.model_dump_json(indent=2))

In [ ]:
"""
{
  "name": "Harry James Potter",
  "age": 37,
  "fact": [
    "He is the chosen one.",
    "He has a lightning-shaped scar on his forehead.",
    "He is the son of James and Lily Potter.",
    "He attended Hogwarts School of Witchcraft and Wizardry.",
    "He is a skilled wizard and sorcerer.",
    "He fought against Lord Voldemort and his followers.",
    "He has a pet owl named Snowy."
  ]
}
"""